In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 라이브러리 로딩

import pandas as pd
import numpy as np
import time
import gc
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 깔끔하게 보기 위해 화면 크기 설정
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 250)

## CSV 파일을 데이터프레임으로 로드

In [ ]:
# 데이터들 읽어오기 

def get_dataset():
    app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
    app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
    prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
    bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
    
    return app_train, app_test, prev, bureau

app_train, app_test, prev, bureau = get_dataset()

### application_train

In [ ]:
# train 데이터 훑어보기

print('application_train data shape : ',app_train.shape)
app_train.head(10)

In [ ]:
app_train.info()

### application_test

In [ ]:
# test 데이터 훑어보기

print('application_test data shape : ',app_test.shape)
app_test.head(10)

 ##### train 데이터와 test 데이터의 모양을 보면 1개 차이가 남 => 예측해야하는 target의 유무때문에 차이가 있는 것입니다.

### bureau

In [ ]:
# bereau 데이터 살펴보기

print('bereau shape : ', bureau.shape)
bureau.head(10)

In [ ]:
bureau.info()

### previous_applications

In [ ]:
# previous_application 데이터 살펴보기

print('previous_applications shape : ', prev.shape)
prev.head(10)

In [ ]:
prev.info()

## EDA (탐색적 데이터분석)

데이터의 추세, 이상, 패턴 또는 관계를 찾기 위한 것으로 데이터가 어떻게 이뤄졌는지 살펴볼 수 있다.

" target은 예측해야 하는 값으로 0은 대출금 상환 가능, 1은 대출금 상환 불가 "

In [ ]:
app_train['TARGET']. value_counts()/app_train.shape[0]

In [ ]:
app_train['TARGET'].astype(int).plot.hist()

train 데이터의 target 비율을 확인해보면 0과 1사이 차이가 매우 크다.
대출금을 제때 상환 가능한 사람들이 대부분임을 알 수 있음

위의 데이터는 불균형한 데이터임을 알 수 있고 이대로 훈련을 하면 정확도가 낮은 예측이 나올것이기 때문에        
여러가지 방법으로 모델링을 한 후 살펴봐야 한다.

In [ ]:
app_train.dtypes.value_counts()

app_train에는 float나 int타입의 numerical feature이 있고 object feature이 있다

### 결측값들 조사및 관찰

In [ ]:
def missing_values(df):
        # 총 결측값의 수
        mis_val = df.isnull().sum()
        
        # 전체 값들중 결측값이 차지하는 비율
        mis_val_percent = round(100 * df.isnull().sum() / len(df), 2)
        
        # 결과 저장 - concat을 이용
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # 열의 이름 수정
        mis_val_table = mis_val_table.rename(
        columns = {0 : '결측값의 수', 1 : '전체대비 결측값'})
        
        # 결측값 내림차순으로 정렬
        mis_val_table_new = mis_val_table[mis_val_table.iloc[:,1] != 0].sort_values('전체대비 결측값', ascending=False)
        
        # 결측값이 있는 column 개수 출력
        print ("해당 데이터프레임이 가진 columns의 수  : " + str(df.shape[1]) + "개\n"      
            "해당 데이터프레임 중 결측값이 있는 columns의 수 " + str(mis_val_table_new.shape[0]) +
              "개")
        
        return mis_val_table_new

In [ ]:
# 결측값 통계내서 살펴보기 - train

missing_values_1 = missing_values(app_train)
print()
missing_values_1

데이터프레임이 가진 column은 총 122개 이며 이들중 결측값이 관측되는 column은 64개이고 위는 결측값의 비율이 높은순으로 정렬한것이다.

In [ ]:
# bureau

missing_values_2 = missing_values(bureau)
print()
missing_values_2

In [ ]:
# previous_applications

missing_values_3 = missing_values(prev)
print()
missing_values_3

나머지 데이터들도 결측값을 훑어봤다.

In [ ]:
# cond1 = TARGET이 1일 경우 (대출상환불가), cond0 = TARGET이 0일경우 (대출상환가능)
cond1 = (app_train['TARGET']==1)
cond0 = (app_train['TARGET']==0)

### 데이터 가공 전 train 데이터와 test 데이터 세트 결합

In [ ]:
apps = pd.concat([app_train, app_test])
apps_o = apps
print(apps_o.shape)

### apps 데이터 살피기

**Binary Feature**

In [ ]:
# 연속형 숫자를 가진 유형의 feature

def show_binary(df, columns):
    for column in columns:
        fig, ax = plt.subplots(figsize=(12,4), nrows=1, ncols=3, squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.kdeplot(df.loc[cond0, column], label='target==0', ax=ax[0][1])
        sns.kdeplot(df.loc[cond1, column], label='target==1', ax=ax[0][1])
        sns.distplot(df[cond0][column], ax=ax[0][2], label='0', color = 'yellow')
        sns.distplot(df[cond1][column], ax=ax[0][2], label='1', color = 'blue')
        
binary_columns =['AMT_INCOME_TOTAL', 'AMT_CREDIT','AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED','CNT_CHILDREN', 'CNT_FAM_MEMBERS']
show_binary(apps, binary_columns)

히스토그램등으로 분포를 확인하면 구간에 따라 결과가 매우 달라질 위험이 있으므로 kdeplot을 사용해서 그래프를 나타냈으며
또 다른 그래프인 violinplot을 이용해서 각 feature들을 살펴보았다.

**AMT_INCOME_TOTAL**, **CNT_CHILDREN**, **CNT_FAM_MEMBERS**은 그래프가 매우 왼쪽으로 치우친 모양이다.
약간의 처리한 후 관찰하도록 하겠습니다.

**AMT_CREDIT** 은 대출금액을 나타내며 다음과 같은 분포입니다.

**AMT_GOODS_PRICE**는 냉장고, 자동차, 컴퓨터 등 고가의 상품을 사기 위해서 대출을 하는 것입니다. 여러 군집으로 나뉘는 것을 볼 수 있습니다.

In [ ]:
cond = apps['AMT_INCOME_TOTAL'] < 400000
sns.violinplot(x='TARGET', y='AMT_INCOME_TOTAL',data=apps[cond])

In [ ]:
# 막대그래프로 target=1의 비율과 해당 조건에 맞는 수를 세어주는 막대그래프 표시

def show_plot(df,Target):
    temp = df[Target].value_counts()
    df1 = pd.DataFrame({Target: temp.index,'Number': temp.values})

    # 카테고리별 target=1일경우의 비율
    cat_perc = df[[Target, 'TARGET']].groupby([Target],as_index=False).mean()
    cat_perc.sort_values(by='TARGET', ascending=False, inplace=True)

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
    sns.set_color_codes("pastel")
    s = sns.barplot(ax=ax1, x = Target, y="Number",data=df1) 
    s = sns.barplot(ax=ax2, x = Target, y='TARGET', order=cat_perc[Target], data=cat_perc)
    plt.ylabel('Percent of target = 1 [%]', fontsize=10)

    plt.show();

In [ ]:
# distplot을 나타내는 함수, 색깔도 지정 가능

def plot_distribution(df, feature, color):
    plt.figure(figsize=(10,6))
    plt.title("Distribution of %s" % feature)
    sns.distplot(df[feature].dropna(),color=color, kde=True ,bins=100)
    plt.show()

In [ ]:
show_plot(apps, 'CNT_CHILDREN')

In [ ]:
show_plot(apps,'CNT_FAM_MEMBERS')

CNT_CHILDREN 과 CNT_FAM_MEMBERS 는 CNT_CHILDREN+2한 분포가 CNT_FAM_MEMBERS의 값의 분포와 유사함으로
CNT_CHILDREN, CNT_FAM_MEMBERS를 같이 묶어 CNT_FAM_MEMBERS로 대체하기로했다.                     
또한 target=1의 비율인 표를 볼때 가족이 많을수록 높은비율로 대출을 갚지 못하는 것을 알 수 있었다

In [ ]:
# DAYS_BIRTH를 나이로 변해서 그래프 나타내기

def show_convert(df, feature):
    df[feature] = abs(df[feature])
    plt.figure(figsize=(10,6))
    sns.kdeplot(df.loc[df['TARGET']==0, feature]/365, label='0', color='yellow')
    sns.kdeplot(df.loc[df['TARGET']==1, feature]/365, label='1')
    plt.xlabel(feature)
    plt.title('Distribution of '+ feature)

In [ ]:
show_convert(apps, 'DAYS_BIRTH')

나이가 어릴수록 노란색 그래프가 더 적은것을 보아 대출을 갚을 능력이 떨어지고 40대 이상부터는 대출을 갚을 확률이 더 높아진다

In [ ]:
show_convert(apps, 'DAYS_EMPLOYED')

DAYS_EMPLOYED에서 **365243**이 자주 나타나는데 365243을 365로 나누면 약 1000으로 천년동안 근무를 하는것은 이상하다.        
따라서 다음의 값은 이상치라고 봐도 무방하다.

**Object Feature**

범주형 변수들은 개별값을 숫자로 바꿔주어야 하는데 범주형 feature을 숫자로 인코딩하는 방법은                         
Label 인코딩과 one-hot 인코딩으로 범주형 변수가 고유의 값을 두개 이상 가진다면 one-hot 인코딩을 권장한다

In [ ]:
# object feature을 object_columns 에 담고 이들을 시각화

object_columns = app_train.dtypes[app_train.dtypes=='object'].index.tolist()

def show_category(df,columns):
    for column in columns:
        # distplot은 연속형값 표현에 쓰는것이기 때문에 주로 catplot을 사용
        chart=sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=60)
        chart.fig.set_figwidth(12)
        chart.fig.set_figheight(2)
        # 그래프의 x축 항목들을 잘 보기위해 수행, 크기조정
        
show_category(app_train, object_columns)

### Feature들간의 상관관계

In [ ]:
# 상관관계순 오름차순 정렬
correlations = app_train.corr()['TARGET'].sort_values()
print('가장 큰 negative correlation 상위 10개:\n', correlations.head(10))
print()
# 상관관계 내림차순 정렬
correlations = app_train.corr()['TARGET'].sort_values(ascending=False)
print('가장 큰 positive correlation 상위 10개:\n', correlations.head(10))

In [ ]:
correlations.count()

122-16=106 이므로 object feature은 상관관계로 표현되지 않는다.

#### **heatmap으로 살펴보도록 하자**

In [ ]:
binarys = np.array(binary_columns)
sns.heatmap(app_train[binarys].corr(), annot=True)
plt.show()

### Bureau data 살피기

**SK_ID_CURR을 기준으로 bureau와 app_train과 합친 후 살펴봐야한다**

In [ ]:
bureau_train = apps.merge(bureau, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
bureau_train_o = bureau_train

#### **Bureau 데이터에서 쓸 피처들 6가지**

화폐 유형, 현재 채무금액, 신용카드 한도, 대출 상태, 신용 연장 횟수, 대출 신청시 CB 크레딧 연체 일수

In [ ]:
b_columns = ['CREDIT_CURRENCY', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'CREDIT_ACTIVE', 'CNT_CREDIT_PROLONG', 'CREDIT_DAY_OVERDUE']

In [ ]:
# bureau_train에서 해당하는 컬럼들만 선택해서 다시 train 데이터를 만들고 숫자형 feature만 골라서 리스트로 만듦

bureau_train=bureau_train[['SK_ID_CURR','TARGET']+b_columns]
b_num_columns = bureau_train.dtypes[bureau_train.dtypes != 'object'].index.tolist()
b_num_columns = b_num_columns[2:6]
print(b_num_columns)

In [ ]:
# bureau_train에서 object feature만 골라서 리스트로 만듦

b_object_columns = bureau_train.dtypes[bureau_train.dtypes == 'object'].index.tolist()
print(b_object_columns)

**Binary feature**

In [ ]:
# object 아닌 유형의 feature 시각화

def show_binary_2(df, columns):
    for column in columns:
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3), squeeze=False)
        
        # bureau는 특정 컬럼값이 infinite로 들어가있는 경우가 있음.  infinite일 경우 KDE histogram 시각화 시 문제 발생하여 이를 제거
        sns.violinplot(x='TARGET', y=column, data=df[np.isfinite(df[column])], ax=ax[0][0])
        sns.distplot(df[cond0 & np.isfinite(df[column])][column], label='0', color='coral', ax=ax[0][1])
        sns.distplot(df[cond1 & np.isfinite(df[column])][column], label='1', color='turquoise', ax=ax[0][1])
        
show_binary_2(bureau_train, b_num_columns)

값들이 너무 쏠려있으므로 조정해주기 위해 통계치를 알아봤다.

In [ ]:
TB0=bureau_train[bureau_train['TARGET']==0]
TB1=bureau_train[bureau_train['TARGET']==1]

In [ ]:
print("현재 채무 금액 >")
round(bureau_train['AMT_CREDIT_SUM_DEBT'].describe(),1)

In [ ]:
print("신용카드 한도 >")
round(bureau_train['AMT_CREDIT_SUM_LIMIT'].describe(),2)

In [ ]:
print("신용 연장 횟수 >")
round(bureau_train['CNT_CREDIT_PROLONG'].describe(),2)

In [ ]:
print("신용 연장횟수에 따른 target=0일 확률", "단, NAN의 경우는 target=1인 경우가 없을때이다. 즉 100%")
TB0['CNT_CREDIT_PROLONG'].value_counts()/(TB0['CNT_CREDIT_PROLONG'].value_counts()+TB1['CNT_CREDIT_PROLONG'].value_counts())*100

**신용 연장횟수는 평균적으로 0번이다.**
다만 놀랍게도 연장횟수가 늘어날수록 target=0일 확률이 올라간다

In [ ]:
print(" 대출 신청시 연체 일수")
round(bureau_train['CREDIT_DAY_OVERDUE'].describe(),2)

In [ ]:
binarys = np.array(b_num_columns)
sns.heatmap(bureau_train[binarys].corr(), annot=True)
plt.show()

**Object feature**

In [ ]:
# object 유형의 feature 시각화

show_category(bureau_train, b_object_columns)

In [ ]:
bureau_train['CREDIT_CURRENCY'].value_counts()

CREDIT_CURRENCY를 보면 currency 1이 99.9%를 차지하고 있으므로 그냥 currency1이 전부를 차지하고 있다고 봐도 무방하므로 해당 항목은 의미가 없다.

CREDIT_ACTIVE는 주로 CLOSED(대출 상환 완료)와 ACTIVE(대출 상환중) 상태로 나뉘며 대출상태를 나타낸다                                                       
**대출 상환 완료 상태일수록 Target=0일 확률이 높다.**

### Previous_application 살피기
**SK_ID_CURR을 기준으로 bureau와 app_train과 합친 후 살펴봐야한다**

In [ ]:
prev_train = apps.merge(prev, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
prev_train_o = prev_train

#### **Previous_application 데이터에서 쓸 피처들 8가지**

대출 유형, 과거대비 현재 신청결정기간, 대출시 납부한 선금액, 대출 신청금액, 대출금액(허가), 대출 시작한 요일, 동행 고객, 대출시 보험 요청여부

In [ ]:
prev_columns=['NAME_CONTRACT_TYPE_x', 'DAYS_DECISION', 'AMT_DOWN_PAYMENT', 'AMT_APPLICATION', 'AMT_CREDIT_x','WEEKDAY_APPR_PROCESS_START_y', 'NAME_TYPE_SUITE_x', 'NFLAG_INSURED_ON_APPROVAL']

In [ ]:
prev_train=prev_train[prev_columns+['SK_ID_CURR', 'SK_ID_PREV','TARGET']]
prev_train.rename(columns={'NAME_CONTRACT_TYPE_x':'NAME_CONTRACT_TYPE', 'AMT_CREDIT_x':'AMT_CREDIT', 'WEEKDAY_APPR_PROCESS_START_y':'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE_x':'NAME_TYPE_SUITE' }, inplace=True)
print(prev_train.dtypes)

In [ ]:
# prev_train에서 해당하는 컬럼들만 선택해서 다시 train 데이터를 만들고 숫자형 feature만 골라서 리스트로 만듦
prev_num_columns = ['DAYS_DECISION', 'AMT_DOWN_PAYMENT', 'AMT_APPLICATION', 'AMT_CREDIT', 'NFLAG_INSURED_ON_APPROVAL']

In [ ]:
# prev_train에서 object feature만 골라서 리스트로 만듦

prev_object_columns = ['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE']

**SK_ID_CURR당 평균 SK_ID_PREV 건수 구하기**

In [ ]:
# groupby를 이용해 평균건수 구하기

mean_prev = prev_train.groupby('SK_ID_CURR')['SK_ID_CURR'].count().mean()
print("SK_ID_CURR당 평균 SK_ID_PREV 건수 : " , round(mean_prev,2))
print()
g=sns.boxplot(prev_train.groupby('SK_ID_CURR')['SK_ID_CURR'].count())

**Binary Feature**

In [ ]:
show_binary_2(prev_train, prev_num_columns)

**DAYS_DECISION**은 TARGET=1일 때 0에 가까운(최근일)값이 약간 더 많음.                                
**AMT_CREDIT, AMT_APPLICATION** 큰차이는 아니지만 TARGET=1일 경우에 소액 비율이 약간 높음                   
**RATE_DOWN_PAYMENT**는 큰 차이 없음.

In [ ]:
binarys = np.array(prev_num_columns)
sns.heatmap(prev_train[binarys].corr(), annot=True)
plt.show()

가장 상관관계가 큰것은 AMT_APPLICATION과 AMT_PAYMENT이다

**Object Feature**

In [ ]:
# object 유형의 feature 시각화

show_category(prev_train, prev_object_columns)

In [ ]:
TP0 = prev_train[prev_train['TARGET']==0]
TP1 = prev_train[prev_train['TARGET']==1]

print("NAME_CONTRACT_TYPE에 TARGET=0일 확률 >> ")
print(TP0['NAME_CONTRACT_TYPE'].value_counts()/(TP0['NAME_CONTRACT_TYPE'].value_counts()+TP1['NAME_CONTRACT_TYPE'].value_counts())*100)

##### => resolving loan일때 확률이 조금 더 높음

In [ ]:
print("NAME_TYPE_SUITE의 분포 비 >> ")
print(prev_train['NAME_TYPE_SUITE'].value_counts()/prev_train['NAME_TYPE_SUITE'].value_counts().sum() * 100)

##### ==> 크게 unaccompanied 와 Family 그리고 나머지로 나눌 수 있다.

## Feature Engineering 수행

**전처리과정도 함께 수행하도록 한다**

**apps**

In [ ]:
# 성별 인코딩
apps['CODE_GENDER'] = pd.factorize(apps['CODE_GENDER'])[0]

In [ ]:
apps['APPS_EXT_SOURCE_MEAN']=apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD']=apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
apps['APPS_EXT_SOURCE_STD'] =apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())

In [ ]:
#AMT_CREDIT 으로 가공
apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/ apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

In [ ]:
# FLAG_OWN_CAR, FLAG_OWN_REALTY 모두 소유할경우, 한경우만 소유할 경우 둘다 소지하지 않을경우

apps['FLAG_OWN_CAR']= pd.factorize(apps['FLAG_OWN_CAR'])[0]
apps['FLAG_OWN_REALTY']=pd.factorize(apps['FLAG_OWN_REALTY'])[0]
apps['FLAG_OWN'] = 0 if (apps['FLAG_OWN_CAR']==0 and apps['FLAG_OWN_REALTY']==0) else (1 if (apps['FLAG_OWN_CAR']==0 or apps['FLAG_OWN_REALTY']==0) else 2)

In [ ]:
apps = apps.fillna(-999)

**Bureau**

In [ ]:
# 현재 크래딧 총액 금액과 채무 총액 금액 사이 비율과 차이에 대한 feature 생성
bureau_train_o['BUREAU_CREDIT_DEBT_RATIO'] = bureau_train_o['AMT_CREDIT_SUM_DEBT'] / bureau_train_o['AMT_CREDIT_SUM']
bureau_train_o['BUREAU_CREDIT_DEBT_DIFF'] = bureau_train_o['AMT_CREDIT_SUM_DEBT'] - bureau_train_o['AMT_CREDIT_SUM']

In [ ]:
# 연채 했을경우의 값들 조사
print(round(bureau_train[bureau_train_o['CREDIT_DAY_OVERDUE']!=0]['CREDIT_DAY_OVERDUE'].describe(),2))
print()
bureau_train_o['CREDIT_DAY_OVERDUE'].value_counts()

In [ ]:
bureau_train_o['CREDIT_DAY_OVERDUE_YN'] = bureau_train_o['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x >0 else 0)
bureau_train_o['CREDIT_DAY_OVERDUE_YN'].value_counts()

연체한 건수보다 연체 안한 건수의 수가 더 많음 따라서 값이 0일때와 아닐때로 변경해준다

**CREDIT_ACTIVE** 는 object feature이므로 라벨인코딩을 수행 0과 1값이 아니면 -999가 되도록 변경

In [ ]:
bureau_train_o['CREDIT_ACTIVE']=pd.factorize(bureau_train_o['CREDIT_ACTIVE'])[0]
bureau_train_o['CREDIT_ACTIVE_NEW'] = bureau_train_o['CREDIT_ACTIVE'].apply(lambda x: -999 if x > 1 else x)
bureau_train_o['CREDIT_ACTIVE_NEW'].value_counts()

In [ ]:
# 위에서 연체 횟수가 5회이상이면 TARGET이 1인 경우는 없으므로 다음의 feature 추가
bureau_train_o['CNT_CREDIT_PROLONG_YN'] = bureau_train_o['CNT_CREDIT_PROLONG'].apply(lambda x: 1 if x >4 else 0)
bureau_train_o['CNT_CREDIT_PROLONG_YN'].value_counts()

In [ ]:
bureau_train_o.drop('CREDIT_CURRENCY', axis=1)

CREDIT_CURRENCY는 별로 큰 영향을 못미치기 때문에 해당 column은 삭제

In [ ]:
bureau_train_o = bureau_train_o.fillna(-999)

**Previous_application**

In [ ]:
# 일수를 년연수로 변경

prev_train_o['MONTH_DECISION'] =  abs(prev_train_o['DAYS_DECISION'])/12
prev_train_o.drop('DAYS_DECISION', axis=1)

print(round(abs(prev_train_o['MONTH_DECISION']).describe(),2))
print(prev_train_o['MONTH_DECISION'].isnull().sum())

In [ ]:
sns.distplot(prev_train_o[prev_train_o['TARGET']==0]['MONTH_DECISION'], label='0', color = 'yellow')
sns.distplot(prev_train_o[prev_train_o['TARGET']==1]['MONTH_DECISION'], label='1', color = 'blue')

In [ ]:
# 대출 신청금액과 허가된 대출 금액 차이

prev_train_o['PREV_CREDIT_DIFF'] = prev_train_o['AMT_APPLICATION'] - prev_train_o['AMT_CREDIT_x']

In [ ]:
# 대출 신청금액과 납부선금액 차이

prev_train_o['AMT_DOWN_DIFF']= prev_train_o['AMT_APPLICATION'] - prev_train_o['AMT_DOWN_PAYMENT']

In [ ]:
# 요일 원핫인코딩
prev_train_o=pd.get_dummies(prev_train_o, columns = ['WEEKDAY_APPR_PROCESS_START_y'])
prev_train_o.head(5)

In [ ]:
# 동행 고객 인코딩

prev_train_o['NAME_TYPE_SUITE_x']=pd.factorize(prev_train_o['NAME_TYPE_SUITE_x'])[0]

In [ ]:
prev_train_o = prev_train_o.fillna(-999)

## 데이터들을 분리하고 LGBM Classifier로 학습 수행

In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

In [ ]:
apps_train = apps[-apps['TARGET'].isnull()]
app_test =apps[apps['TARGET'].isnull()]
bureau_trains = bureau_train_o[- bureau_train_o['TARGET'].isnull()]
prev_trains = prev_train_o[-prev_train_o['TARGET'].isnull()]

ftr_app1=apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
ftr_app2=bureau_trains.drop(['SK_ID_CURR', 'TARGET'], axis=1)
ftr_app3=prev_trains.drop(['SK_ID_CURR', 'TARGET'], axis=1)
ftr_app = app1.merge(app2, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
ftr_app = ftr_app.merge(app3, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
ftr_app = ftr_app.drop(['SK_ID_CURR', 'TARGET'], axis=1)

t1 = apps_train['SK_ID_CURR', 'TARGET']
t2 = bureau_trains['SK_ID_CURR', 'TARGET']
t3 = prev_trains['SK_ID_CURR', 'TARGET']
target_app = t1.mearge(t2, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
target_app = target_app.mearge(t3, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
target_app =target_app.drop(['SK_ID_CURR'], axis=1)

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)

In [ ]:
clf=LGBMClassifier(
    n_jobs=-1,
    n_estimators=1000,
    learning_rate=0.02
    num_leaves=32
    subsample=0.8
    max_depth=12
    silent=-1
    verbose=-1 
)

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=100, early_stopping_rounds=100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsie = (15,30))

In [ ]:
clf.predict_probal(app_Test.drop(['SK_ID_CURR'], axis=1))[:,1]

app_test['TARGET'] =pred
app_test[['SK_ID_CURR', 'TARGET']].to_csv('app_baseline.csv', index=False)